In [1]:
!pip -q install qiskit
!pip -q install qiskit-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.4/643.4 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 19.7 MB/s eta 0:00:00


In [31]:
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import StatevectorSampler, Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer

from docplex.mp.model import Model
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import QuadraticProgramToQubo
import numpy as np

## ILP Formulation of BPP using Docplex

In [3]:
def bin_packing_bpp(sizes, bin_capacity):

    n = len(sizes)       # Number of items
    m = n                # Upper bound on number of bins (at most one item per bin in the worst case)

    model = Model("Bin_Packing")

    x = {(i, j): model.binary_var(name=f"x_{i}_{j}") for i in range(n) for j in range(m)}
    y = {j: model.binary_var(name=f"y_{j}") for j in range(m)}

    model.minimize(model.sum(y[j] for j in range(m)))

    for i in range(n):
        model.add_constraint(model.sum(x[i, j] for j in range(m)) == 1, ctname=f"assign_item_{i}")

    for j in range(m):
        model.add_constraint(
            model.sum(sizes[i] * x[i, j] for i in range(n)) <= bin_capacity * y[j],
            ctname=f"bin_capacity_{j}"
        )

    return model

In [4]:
sizes = [3, 2, 1]
bin_capacity = 3

docplex_model = bin_packing_bpp(sizes, bin_capacity)
docplex_model.prettyprint()

// This file has been generated by DOcplex
// model name is: Bin_Packing
// single vars section
dvar bool x_0_0;
dvar bool x_0_1;
dvar bool x_0_2;
dvar bool x_1_0;
dvar bool x_1_1;
dvar bool x_1_2;
dvar bool x_2_0;
dvar bool x_2_1;
dvar bool x_2_2;
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;

minimize
 y_0 + y_1 + y_2;
 
subject to {
 assign_item_0:
  x_0_0 + x_0_1 + x_0_2 == 1;
 assign_item_1:
  x_1_0 + x_1_1 + x_1_2 == 1;
 assign_item_2:
  x_2_0 + x_2_1 + x_2_2 == 1;
 bin_capacity_0:
  3 x_0_0 + 2 x_1_0 + x_2_0 <= 3 y_0;
 bin_capacity_1:
  3 x_0_1 + 2 x_1_1 + x_2_1 <= 3 y_1;
 bin_capacity_2:
  3 x_0_2 + 2 x_1_2 + x_2_2 <= 3 y_2;

}


## Transforming the ILP model into a QUBO

In [5]:
def ILP_to_QUBO(ILP):

  qp = from_docplex_mp(ILP)
  print("QuadraticProgram obtained from docplex")
  print(qp.prettyprint())

  converter = QuadraticProgramToQubo()
  print(converter.get_compatibility_msg(qp))
  qubo_bpp = converter.convert(qp)

  return qubo_bpp

QuadraticProgram obtained from docplex
Problem name: Bin_Packing

Minimize
  y_0 + y_1 + y_2

Subject to
  Linear constraints (6)
    x_0_0 + x_0_1 + x_0_2 == 1  'assign_item_0'
    x_1_0 + x_1_1 + x_1_2 == 1  'assign_item_1'
    x_2_0 + x_2_1 + x_2_2 == 1  'assign_item_2'
    3*x_0_0 + 2*x_1_0 + x_2_0 - 3*y_0 <= 0  'bin_capacity_0'
    3*x_0_1 + 2*x_1_1 + x_2_1 - 3*y_1 <= 0  'bin_capacity_1'
    3*x_0_2 + 2*x_1_2 + x_2_2 - 3*y_2 <= 0  'bin_capacity_2'

  Binary variables (12)
    x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2 y_0 y_1 y_2



In [51]:
qubo_bpp = ILP_to_QUBO(docplex_model)
print(qubo_bpp.prettyprint())

Problem name: Bin_Packing

Minimize
  4*bin_capacity_0@int_slack@0^2
  + 16*bin_capacity_0@int_slack@0*bin_capacity_0@int_slack@1
  + 16*bin_capacity_0@int_slack@1^2 + 4*bin_capacity_1@int_slack@0^2
  + 16*bin_capacity_1@int_slack@0*bin_capacity_1@int_slack@1
  + 16*bin_capacity_1@int_slack@1^2 + 4*bin_capacity_2@int_slack@0^2
  + 16*bin_capacity_2@int_slack@0*bin_capacity_2@int_slack@1
  + 16*bin_capacity_2@int_slack@1^2 + 24*x_0_0*bin_capacity_0@int_slack@0
  + 48*x_0_0*bin_capacity_0@int_slack@1 + 40*x_0_0^2 + 8*x_0_0*x_0_1
  + 8*x_0_0*x_0_2 + 48*x_0_0*x_1_0 + 24*x_0_0*x_2_0 - 72*x_0_0*y_0
  + 24*x_0_1*bin_capacity_1@int_slack@0 + 48*x_0_1*bin_capacity_1@int_slack@1
  + 40*x_0_1^2 + 8*x_0_1*x_0_2 + 48*x_0_1*x_1_1 + 24*x_0_1*x_2_1 - 72*x_0_1*y_1
  + 24*x_0_2*bin_capacity_2@int_slack@0 + 48*x_0_2*bin_capacity_2@int_slack@1
  + 40*x_0_2^2 + 48*x_0_2*x_1_2 + 24*x_0_2*x_2_2 - 72*x_0_2*y_2
  + 16*x_1_0*bin_capacity_0@int_slack@0 + 32*x_1_0*bin_capacity_0@int_slack@1
  + 20*x_1_0^2 + 8*x_1

## Solving QUBO using QAOA

In [9]:
op, offset = qubo_bpp.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

offset: 205.5
operator:
SparsePauliOp(['IIIIIIIIZIIIIIIIII', 'IIIIIIIZIIIIIIIIII', 'IIIIIIZIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIZIIIIII', 'IIIIIIIIIIZIIIIIII', 'IIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIZIZ', 'IIIIIIIIIIIIIIZIIZ', 'IIIIIIIIIIIZIIIIIZ', 'IIIIIIIIZIIIIIIIIZ', 'IIIIIZIIIIIIIIIIIZ', 'IIIIIZIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIIZ', 'IIIIZIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIZIIZI', 'IIIIIIIIIIZIIIIIZI', 'IIIIIIIZIIIIIIIIZI', 'IIIZIIIIIIIIIIIIZI', 'IIIZIIIIIIIIIIIIII', 'IIZIIIIIIIIIIIIIZI', 'IIZIIIIIIIIIIIIIII', 'IIIIIIIIIIIIZIIZII', 'IIIIIIIIIZIIIIIZII', 'IIIIIIZIIIIIIIIZII', 'IZIIIIIIIIIIIIIZII', 'IZIIIIIIIIIIIIIIII', 'ZIIIIIIIIIIIIIIZII', 'ZIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIZZIII', 'IIIIIIIIIIIIZIZIII', 'IIIIIIIIIIIZIIZIII', 'IIIIIIIIZIIIIIZIII', 'IIIIIZIIIIIIIIZIII', 'IIIIZIIIIIIIIIZIII', 'IIIIIIIIIIIIZZIIII', 'IIIIIIIIIIZIIZ

In [32]:
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0.0, 0.0])
exact_mes = NumPyMinimumEigensolver()

<ipython-input-32-4da92eebfbf9>:2: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0.0, 0.0])


In [34]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [29]:
exact_result = exact.solve(qubo_bpp)
print(exact_result.prettyprint())

objective function value: 2.0
variable values: x_0_0=0.0, x_0_1=1.0, x_0_2=0.0, x_1_0=0.0, x_1_1=0.0, x_1_2=1.0, x_2_0=0.0, x_2_1=0.0, x_2_2=1.0, y_0=0.0, y_1=1.0, y_2=1.0, bin_capacity_0@int_slack@0=0.0, bin_capacity_0@int_slack@1=0.0, bin_capacity_1@int_slack@0=0.0, bin_capacity_1@int_slack@1=0.0, bin_capacity_2@int_slack@0=0.0, bin_capacity_2@int_slack@1=0.0
status: SUCCESS


In [35]:
qaoa_result = qaoa.solve(qubo_bpp)
print(qaoa_result.prettyprint())

objective function value: 2.0
variable values: x_0_0=0.0, x_0_1=1.0, x_0_2=0.0, x_1_0=1.0, x_1_1=0.0, x_1_2=0.0, x_2_0=1.0, x_2_1=0.0, x_2_2=0.0, y_0=1.0, y_1=1.0, y_2=0.0, bin_capacity_0@int_slack@0=0.0, bin_capacity_0@int_slack@1=0.0, bin_capacity_1@int_slack@0=0.0, bin_capacity_1@int_slack@1=0.0, bin_capacity_2@int_slack@0=0.0, bin_capacity_2@int_slack@1=0.0
status: SUCCESS
